In [ ]:
import cv2
import os
import shutil
from tqdm import tqdm
import numpy as np
import random

# ====== Cấu hình ======
INPUT_DIR = "D:/KLTN/SKINTONE/public/dataset/train_dataset"
OUTPUT_DIR = "D:/KLTN/SKINTONE/public/dataset/final_data_train"
os.makedirs(OUTPUT_DIR, exist_ok=True)

TYPES = ["Type_1", "Type_2", "Type_3", "Type_4", "Type_5", "Type_6"]
GRAY_COLOR = (128, 128, 128)
TARGET_IMAGES = 3600
AUGMENT_ANGLES = [-15, -10, -5, 5, 10, 15]

# ====== Bắt đầu xử lý ======
for tone in TYPES:
    input_path = os.path.join(INPUT_DIR, tone)
    output_path = os.path.join(OUTPUT_DIR, tone)
    os.makedirs(output_path, exist_ok=True)

    if not os.path.exists(input_path):
        continue

    image_list = [f for f in os.listdir(input_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    num_original = len(image_list)
    num_needed = TARGET_IMAGES - num_original
    print(f"{tone}: {num_original} ảnh gốc, cần thêm {num_needed} ➜ tổng ~{TARGET_IMAGES}")

    # --- Lưu ảnh gốc ---
    for img_name in tqdm(image_list, desc=f"Sao chép gốc {tone}"):
        src = os.path.join(input_path, img_name)
        dst = os.path.join(output_path, img_name)
        shutil.copy(src, dst)

    # --- Nếu cần augment thêm ---
    if num_needed > 0:
        augment_per_image = max(1, num_needed // num_original + 1)

        for img_name in tqdm(image_list, desc=f"Tăng cường {tone}"):
            img_path = os.path.join(input_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue

            h, w = img.shape[:2]
            base_name = os.path.splitext(img_name)[0]

            # Tạo nền xám lớn
            max_dim = int(max(h, w) * 1.5)
            gray_bg = np.full((max_dim, max_dim, 3), GRAY_COLOR, dtype=np.uint8)
            x_offset = (max_dim - w) // 2
            y_offset = (max_dim - h) // 2
            gray_bg[y_offset:y_offset+h, x_offset:x_offset+w] = img

            used_angles = set()
            for i in range(augment_per_image):
                if len(used_angles) >= len(AUGMENT_ANGLES):
                    break  # Không xoay trùng 1 ảnh quá nhiều lần

                # Chọn góc chưa dùng
                angle = random.choice([a for a in AUGMENT_ANGLES if a not in used_angles])
                used_angles.add(angle)

                M = cv2.getRotationMatrix2D((max_dim // 2, max_dim // 2), angle, 1.0)
                rotated = cv2.warpAffine(gray_bg, M, (max_dim, max_dim), borderValue=GRAY_COLOR)
                crop = rotated[y_offset:y_offset+h, x_offset:x_offset+w]

                new_name = f"{base_name}_rot{angle:+}.jpg"
                save_path = os.path.join(output_path, new_name)
                cv2.imwrite(save_path, crop)

                # Kiểm tra đủ ảnh chưa
                if len(os.listdir(output_path)) >= TARGET_IMAGES:
                    break

            if len(os.listdir(output_path)) >= TARGET_IMAGES:
                break

    print(f"{tone} - Tổng ảnh sau augment: {len(os.listdir(output_path))}")

print("\n Hoàn tất cân bằng toàn bộ 6 tone!")


🎯 Type_1: 548 ảnh gốc, cần thêm 3052 ➜ tổng ~3600


Tăng cường Type_1:  93%|█████████▎| 508/548 [00:08<00:00, 60.15it/s]


✅ Type_1 - Tổng ảnh sau augment: 3600
🎯 Type_2: 1785 ảnh gốc, cần thêm 1815 ➜ tổng ~3600


Tăng cường Type_2:  51%|█████     | 907/1785 [00:06<00:06, 135.22it/s]


✅ Type_2 - Tổng ảnh sau augment: 3600
🎯 Type_3: 243 ảnh gốc, cần thêm 3357 ➜ tổng ~3600


...

✅ Type_4 - Tổng ảnh sau augment: 3600
🎯 Type_5: 3598 ảnh gốc, cần thêm 2 ➜ tổng ~3600


Tăng cường Type_5:   0%|          | 1/3598 [00:00<00:50, 71.55it/s]


✅ Type_5 - Tổng ảnh sau augment: 3600
🎯 Type_6: 1693 ảnh gốc, cần thêm 1907 ➜ tổng ~3600


Tăng cường Type_6:  56%|█████▋    | 953/1693 [00:08<00:06, 106.06it/s]

✅ Type_6 - Tổng ảnh sau augment: 3600

🎉 Hoàn tất cân bằng toàn bộ 6 tone!


In [6]:
import os
import random

# === Cấu hình ===
TONE_DIR = "D:/KLTN/SKINTONE/public/balanced_data_gray/Type_3"
KEEP_IMAGES = 2100  # Giữ lại 2000 ảnh
IMAGE_EXT = ('.jpg', '.jpeg', '.png', '.bmp')

# === Lọc danh sách ảnh ===
image_files = [f for f in os.listdir(TONE_DIR) if f.lower().endswith(IMAGE_EXT)]
total = len(image_files)
print(f"📦 Tổng ảnh trong Type_3: {total}")

if total <= KEEP_IMAGES:
    print("✅ Số ảnh đã phù hợp, không cần giảm.")
else:
    # Chọn ngẫu nhiên KEEP_IMAGES ảnh để giữ
    keep_list = set(random.sample(image_files, KEEP_IMAGES))

    # Xóa những ảnh không nằm trong danh sách giữ
    removed = 0
    for img_name in image_files:
        if img_name not in keep_list:
            os.remove(os.path.join(TONE_DIR, img_name))
            removed += 1

    print(f"✅ Đã xóa {removed} ảnh, giữ lại {KEEP_IMAGES} ảnh.")


📦 Tổng ảnh trong Type_3: 2824
✅ Đã xóa 724 ảnh, giữ lại 2100 ảnh.


In [7]:
import os
import random
import shutil

# ========== CẤU HÌNH ==========
BASE_DIR = "D:/KLTN/SKINTONE/public/balanced_data_gray"
TYPES_TO_REDUCE = {
    "Type_1": 2100,
    "Type_5": 2100,
    "Type_6": 2100,
}
IMAGE_EXT = ('.jpg', '.jpeg', '.png', '.bmp')

# ========== XỬ LÝ ==========
for tone, keep_count in TYPES_TO_REDUCE.items():
    tone_dir = os.path.join(BASE_DIR, tone)
    extra_dir = os.path.join(BASE_DIR, f"{tone}_extra")
    os.makedirs(extra_dir, exist_ok=True)

    if not os.path.exists(tone_dir):
        print(f"⚠️ Không tìm thấy thư mục: {tone_dir}")
        continue

    image_files = [f for f in os.listdir(tone_dir) if f.lower().endswith(IMAGE_EXT)]
    total = len(image_files)
    print(f"\n📂 {tone}: {total} ảnh")

    if total <= keep_count:
        print(f"✅ Đã đủ hoặc ít hơn {keep_count} ảnh, không cần giảm.")
        continue

    # Chọn ngẫu nhiên ảnh để giữ
    keep_files = set(random.sample(image_files, keep_count))

    moved = 0
    for img_name in image_files:
        src = os.path.join(tone_dir, img_name)
        if img_name not in keep_files:
            dst = os.path.join(extra_dir, img_name)
            shutil.move(src, dst)
            moved += 1

    print(f"✅ Đã giữ {keep_count} ảnh, di chuyển {moved} ảnh dư vào: {extra_dir}")



📂 Type_1: 2336 ảnh
✅ Đã giữ 2100 ảnh, di chuyển 236 ảnh dư vào: D:/KLTN/SKINTONE/public/balanced_data_gray\Type_1_extra

📂 Type_5: 2132 ảnh
✅ Đã giữ 2100 ảnh, di chuyển 32 ảnh dư vào: D:/KLTN/SKINTONE/public/balanced_data_gray\Type_5_extra

📂 Type_6: 2164 ảnh
✅ Đã giữ 2100 ảnh, di chuyển 64 ảnh dư vào: D:/KLTN/SKINTONE/public/balanced_data_gray\Type_6_extra


In [2]:
import os
import cv2
import numpy as np
import random
from tqdm import tqdm

# ====== CẤU HÌNH ======
INPUT_DIR = "D:/KLTN/SKINTONE/public/data_4/dataset_train/train"
TYPES = ["Type_1", "Type_2", "Type_3", "Type_4", "Type_5", "Type_6"]
TARGET_IMAGES = 3600
GRAY_COLOR = (128, 128, 128)
ROTATE_ANGLES = [-15, -10, -5, 5, 10, 15]

# ====== TIỆN ÍCH ======
def is_image_file(f):
    return f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp"))

def random_brightness_contrast(img):
    alpha = random.uniform(0.8, 1.2)
    beta = random.randint(-15, 15)
    return cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

def augment_image(img, h, w):
    max_dim = int(max(h, w) * 1.5)
    gray_bg = np.full((max_dim, max_dim, 3), GRAY_COLOR, dtype=np.uint8)
    x_offset = (max_dim - w) // 2
    y_offset = (max_dim - h) // 2
    gray_bg[y_offset:y_offset+h, x_offset:x_offset+w] = img

    angle = random.choice(ROTATE_ANGLES)
    M = cv2.getRotationMatrix2D((max_dim // 2, max_dim // 2), angle, 1.0)
    rotated = cv2.warpAffine(gray_bg, M, (max_dim, max_dim), borderValue=GRAY_COLOR)
    crop = rotated[y_offset:y_offset+h, x_offset:x_offset+w]

    if random.random() < 0.5:
        crop = cv2.flip(crop, 1)
    if random.random() < 0.5:
        crop = random_brightness_contrast(crop)

    return crop, angle

# ====== TĂNG CƯỜNG ẢNH TRÊN TẬP TRAIN ======
for tone in TYPES:
    tone_path = os.path.join(INPUT_DIR, tone)
    if not os.path.isdir(tone_path):
        continue

    image_list = [f for f in os.listdir(tone_path) if is_image_file(f)]
    print(f"🎯 {tone}: {len(image_list)} ảnh ban đầu")

    if len(image_list) >= TARGET_IMAGES:
        print(f"✅ {tone} đã đủ {len(image_list)} ảnh, không cần augment thêm.")
        continue

    print(f"➕ Augment {tone} để đạt {TARGET_IMAGES} ảnh...")

    augment_count = 0
    while len(os.listdir(tone_path)) < TARGET_IMAGES:
        img_name = random.choice(image_list)
        img_path = os.path.join(tone_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue

        h, w = img.shape[:2]
        aug_img, angle = augment_image(img, h, w)
        base_name = os.path.splitext(img_name)[0]
        save_name = f"{base_name}_aug{augment_count}_rot{angle:+}.jpg"
        save_path = os.path.join(tone_path, save_name)

        if not os.path.exists(save_path):  # tránh ghi đè
            cv2.imwrite(save_path, aug_img)
            augment_count += 1

    print(f"✅ {tone}: Tổng ảnh sau augment = {len(os.listdir(tone_path))}")

print("\n🎉 Đã hoàn tất augment đủ 3600 ảnh cho mỗi tone (chỉ áp dụng train)!")


🎯 Type_1: 141 ảnh ban đầu
➕ Augment Type_1 để đạt 3600 ảnh...
✅ Type_1: Tổng ảnh sau augment = 3600
🎯 Type_2: 993 ảnh ban đầu
➕ Augment Type_2 để đạt 3600 ảnh...
✅ Type_2: Tổng ảnh sau augment = 3600
🎯 Type_3: 317 ảnh ban đầu
➕ Augment Type_3 để đạt 3600 ảnh...
✅ Type_3: Tổng ảnh sau augment = 3600
🎯 Type_4: 761 ảnh ban đầu
➕ Augment Type_4 để đạt 3600 ảnh...
✅ Type_4: Tổng ảnh sau augment = 3600
🎯 Type_5: 482 ảnh ban đầu
➕ Augment Type_5 để đạt 3600 ảnh...
✅ Type_5: Tổng ảnh sau augment = 3600
🎯 Type_6: 81 ảnh ban đầu
➕ Augment Type_6 để đạt 3600 ảnh...
✅ Type_6: Tổng ảnh sau augment = 3600

🎉 Đã hoàn tất augment đủ 3600 ảnh cho mỗi tone (chỉ áp dụng train)!
